In [1]:
# The idea for the exercise was to create a function that given an input pattern
# corresponding to the appropriate pattern for calculating the n-th entry of the signal, 
# updates it to the appropriate pattern for entry n+1.
# The offset and base pattern itself are provided inside the function.
# The function receives the old pattern, the index of the current signal entry (entry n+1 above),
# and the length of the signal
def update_pattern(pattern, output_index,  length):
    
    base_pattern = [0, 1, 0, -1]
    offset = 1
    
    # If calculating first entry, repeat the pattern x times and keep only
    # relevant part (from offset to length of signal)
    if output_index == 0:
        r = base_pattern*length
        pattern[:] = r[offset:length+1]
        return pattern
    
    else:
        pattern.insert(0, 0)
        
        
        for i in range(length):
            # Get element from the base pattern
            elem = base_pattern[(i+1)%4]
            # Get index to insert value (this was kinda found with trial and error...)
            index = output_index*(2+i) + i
            # Break if we are out of range
            if index > length:
                break
            pattern.insert(index, elem)
            
        # pop elements from pattern until it is of the right length
        while len(pattern) > length:
            pattern.pop()
        
        return pattern

In [2]:
with open('input.txt', 'r') as infile:                                    
    raw_signal = list(infile.read().replace('\n', ''))


received = list([int(num) for num in raw_signal])
signal_length = len(received)
signal = received
phases = 100
pattern = []

for repeat in range(phases):
    
    sig_output = []
    for i in range(signal_length):
        # Update pattern
        pattern = update_pattern(pattern, i, signal_length)
        # Calculate corresponding output 
        output = int(str(sum([pattern[j]*signal[j] for j in range(signal_length)]))[-1])
        # Append to result
        sig_output.append(output)

    signal = sig_output
    
print(sig_output[:8])

[7, 4, 3, 6, 9, 0, 3, 3]


### Part 2

In [35]:
# Repeating the input ten thousands times
full_message = raw_signal*10000

print('The length ofthe input signal is',len(full_message))
print('The final message offset is given by the first 7 digits of the input signal, i.e.:', ''.join(raw_signal[:7]))

The length ofthe input signal is 6500000
The final message offset is given by the first 7 digits of the input signal, i.e.: 5976733


With these two information we can see that there is an easy way into the solution. Here's how. 

Let's first take as an example an input like '1234567', and draw a table that represent the pattern for calculating each output in our procedure. We forget about the offset for now.

| digit index | 1 | 2 | 3 | 4 | 5 | 6 | 7 |
| :- |:---|:---|:---|:---|:---|:---|:---|
| $\textbf{0}$ | 1 | 0 | -1 | 0 | 1 | 0 | -1 |
| $\textbf{1}$ | 0 | 1 | 1 | 0 | 0 | 1 | 1 |
| $\textbf{2}$ | 0 | 0 | 1 | 1 | 1 | 0 | 0 |
| $\textbf{3}$ | 0 | 0 | 0 | 1 | 1 | 1 | 1 |
| $\textbf{4}$ | 0 | 0 | 0 | 0 | 1 | 1 | 1 |
| $\textbf{5}$ | 0 | 0 | 0 | 0 | 0 | 1 | 1 |
| $\textbf{6}$ | 0 | 0 | 0 | 0 | 0 | 0 | 1 |

As we can see the $\textbf{last}$ digit will always remain the same, the second-to-last will be just given by the sum of the last 2 digits (and then taking the 10's digit), and so on until we reach the middle-position digit. If we work only the second half of the signal, the pattern is simpler than the full one `[0, 1, 0, -1]`.

In our case, the output signal offset (given by the input signal first 7 digits) is 5976733 and it is well above the 'midpoint' of the signal which has 6500000 digits. This means that we can restric our signal to a relevant part that starts at the first digit after the offset and ends at the end of the message. The pattern corresponding to this first digit is made of only $1$s, so it is basically the $\textbf{sum all the elements}$ of this modified signal. The pattern for the second digit is the same but starts with a zero, so it is the sum minus the previous digit in the signal, and so forth for the other digits. By operating in reverse order we avoid having to sum the elements just for subtracting them after.

The following code runs in about 40 seconds.


In [36]:
offset = int(''.join(raw_signal[:7]))
# Keeping only digits after offset, in reverse order
relevant_signal = [int(x) for x in reversed(full_message[offset:])]

for repeat in range(phases):
    
    output = []
    tot_sum = 0

    for item in relevant_signal:
        tot_sum += item
        new_output = int(str(tot_sum)[-1])
        output.append(new_output)
    
    relevant_signal = output
    
print('The final message is', ''.join([str(x) for x in reversed(relevant_signal[-8:])])) #78725270

The final message is 19903864
